# DIY Covid-19 Dashboard

In [176]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [177]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [178]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata = {}
with open("covidData.json") as covidData:
    jsondata = json.load(covidData)

In [179]:

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe."""

    datalist=rawdata['data']

    # extracts dates from datalist
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()

    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object           """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    # creates index from start and end date
    index=pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['newCases',
     'newTests', 'newAdmissions','covidOccupiedMVBeds'])

    # filles in dataframe with values
    for entry in datalist: 
        date=parse_date(entry['date'])
        for column in ['newCases', 'newTests', 'newAdmissions',
        'covidOccupiedMVBeds']:
            # check that nothing is there yet
            if pd.isna(timeseriesdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                timeseriesdf.loc[date, column]=value
                
    # fill in any remaining "holes" due to missing dates
    timeseriesdf.fillna(0.0, inplace=True)

    return timeseriesdf

df=wrangle_data(jsondata) # df is the dataframe for plotting

In [181]:

def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=nation',
        'areaName=england'
    ]

    structure = {
        "date": "date",
        "newCases": "newCasesByPublishDate",
        "newTests": "newTestsByPublishDate",
        "newAdmissions": "newAdmissions",
        "covidOccupiedMVBeds": "covidOccupiedMVBeds"
    }

    api = Cov19API(filters=filters, structure=structure)
    try:
        data = api.get_json()
    except:
        with outError:
            outError.clear_output()
            print("Data is currently not available")

    return data


In [182]:

def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """

    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    refresh_graph()

    apibutton.icon="check"

    out.clear_output()
    with out:
        print('Last update was on: ', apidata['lastUpdate'])

    
apibutton=wdg.Button(
    description='Fetch data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Fetch new covid data from PHE",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)

apibutton.on_click(api_button_callback)


The graph shows the number of cases of Covid19 positive tests, number of tests undertaken, hospital admissions and number of occupied beds with mechanical ventilation per day in England. 

You can control which stats to display by making a selection in data control and choose the scale between logarithmic and linear.

You can also fetch the latest data from Publich Health England by using the 'Fetch data' button.

In [183]:
series=wdg.SelectMultiple(
    options=['newCases', 'newTests', 'newAdmissions','covidOccupiedMVBeds'], # options available
    value=['newCases', 'newAdmissions'], # initial value
    rows=4, # rows of the selection box
    description='Data',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    value='log', 
    description='Scale:',
    disabled=False
)

# displays the last update date and time
out = wdg.Output()

# Output for errors
outError = wdg.Output()

controls=wdg.HBox([series, scale])
fetchBox = wdg.HBox([apibutton, out])

def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=scale.value
    if current==scale.options[0]:
        other=series.options[1]
    else:
        other=scale.options[0]
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back
    
    
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(outError)
display(fetchBox)
display(controls, graph)

Output()

Output()

**(C) Nahid Topalovic, 2020. All rights reserved.** *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*